# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 7 2023 8:30AM,256761,10,PRF-52091,Bio-PRF,Released
1,Feb 7 2023 8:30AM,256761,10,PRF-52096,Bio-PRF,Released
2,Feb 7 2023 8:30AM,256761,10,PRF-52102,Bio-PRF,Released
3,Feb 7 2023 8:30AM,256761,10,PRF-52105,Bio-PRF,Released
4,Feb 7 2023 8:30AM,256761,10,PRF-52108,Bio-PRF,Released
5,Feb 7 2023 8:30AM,256761,10,PRF-52111,Bio-PRF,Released
6,Feb 7 2023 8:30AM,256761,10,PRF-52114,Bio-PRF,Released
7,Feb 6 2023 3:09PM,256760,19,ADV80011433,"AdvaGen Pharma, Ltd",Released
8,Feb 6 2023 3:09PM,256760,19,ADV80011434,"AdvaGen Pharma, Ltd",Released
9,Feb 6 2023 3:09PM,256760,19,ADV80011435,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,256753,Executing,1
27,256757,Executing,2
28,256757,Released,16
29,256760,Released,21
30,256761,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256750,NaN,NaN,20.0
256753,NaN,1.0,NaN
256757,NaN,2.0,16.0
256760,NaN,NaN,21.0
256761,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256676,0.0,0.0,2.0
256683,0.0,1.0,0.0
256684,0.0,0.0,1.0
256685,0.0,0.0,1.0
256686,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256676,0,0,2
256683,0,1,0
256684,0,0,1
256685,0,0,1
256686,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256676,0,0,2
1,256683,0,1,0
2,256684,0,0,1
3,256685,0,0,1
4,256686,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256676,,,2
1,256683,,1,
2,256684,,,1
3,256685,,,1
4,256686,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 7 2023 8:30AM,256761,10,Bio-PRF
7,Feb 6 2023 3:09PM,256760,19,"AdvaGen Pharma, Ltd"
28,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC"
46,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation
66,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation
67,Feb 6 2023 2:08PM,256748,16,TASA USA Inc.
68,Feb 6 2023 1:20PM,256742,15,"Alliance Pharma, Inc."
73,Feb 6 2023 1:16PM,256740,19,"Emersa Waterbox, LLC"
74,Feb 6 2023 1:15PM,256738,10,"ClearSpec, LLC"
75,Feb 6 2023 1:15PM,256735,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 7 2023 8:30AM,256761,10,Bio-PRF,,,7
1,Feb 6 2023 3:09PM,256760,19,"AdvaGen Pharma, Ltd",,,21
2,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",,2,16
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,,,20
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,,1,
5,Feb 6 2023 2:08PM,256748,16,TASA USA Inc.,,,1
6,Feb 6 2023 1:20PM,256742,15,"Alliance Pharma, Inc.",,,5
7,Feb 6 2023 1:16PM,256740,19,"Emersa Waterbox, LLC",,,1
8,Feb 6 2023 1:15PM,256738,10,"ClearSpec, LLC",,,1
9,Feb 6 2023 1:15PM,256735,15,"Brookfield Pharmaceuticals, LLC",1,18,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:30AM,256761,10,Bio-PRF,7,,
1,Feb 6 2023 3:09PM,256760,19,"AdvaGen Pharma, Ltd",21,,
2,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",16,2,
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,20,,
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,,1,
5,Feb 6 2023 2:08PM,256748,16,TASA USA Inc.,1,,
6,Feb 6 2023 1:20PM,256742,15,"Alliance Pharma, Inc.",5,,
7,Feb 6 2023 1:16PM,256740,19,"Emersa Waterbox, LLC",1,,
8,Feb 6 2023 1:15PM,256738,10,"ClearSpec, LLC",1,,
9,Feb 6 2023 1:15PM,256735,15,"Brookfield Pharmaceuticals, LLC",5,18,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:30AM,256761,10,Bio-PRF,7,,
1,Feb 6 2023 3:09PM,256760,19,"AdvaGen Pharma, Ltd",21,,
2,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",16,2,
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,20,,
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:30AM,256761,10,Bio-PRF,7.0,NaN,NaN
1,Feb 6 2023 3:09PM,256760,19,"AdvaGen Pharma, Ltd",21.0,NaN,NaN
2,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",16.0,2.0,NaN
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,20.0,NaN,NaN
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:30AM,256761,10,Bio-PRF,7.0,0.0,0.0
1,Feb 6 2023 3:09PM,256760,19,"AdvaGen Pharma, Ltd",21.0,0.0,0.0
2,Feb 6 2023 2:29PM,256757,10,"Digital Brands, LLC",16.0,2.0,0.0
3,Feb 6 2023 2:14PM,256750,10,ISDIN Corporation,20.0,0.0,0.0
4,Feb 6 2023 2:11PM,256753,10,ISDIN Corporation,0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3080783,185.0,18.0,0.0
12,513428,8.0,1.0,0.0
15,770172,12.0,18.0,33.0
16,770139,2.0,1.0,0.0
19,770176,24.0,0.0,0.0
21,770052,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3080783,185.0,18.0,0.0
1,12,513428,8.0,1.0,0.0
2,15,770172,12.0,18.0,33.0
3,16,770139,2.0,1.0,0.0
4,19,770176,24.0,0.0,0.0
5,21,770052,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,185.0,18.0,0.0
1,12,8.0,1.0,0.0
2,15,12.0,18.0,33.0
3,16,2.0,1.0,0.0
4,19,24.0,0.0,0.0
5,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,185.0
1,12,Released,8.0
2,15,Released,12.0
3,16,Released,2.0
4,19,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,0.0,0.0,33.0,0.0,0.0,0.0
Executing,18.0,1.0,18.0,1.0,0.0,1.0
Released,185.0,8.0,12.0,2.0,24.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,33.0,0.0,0.0,0.0
1,Executing,18.0,1.0,18.0,1.0,0.0,1.0
2,Released,185.0,8.0,12.0,2.0,24.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,33.0,0.0,0.0,0.0
1,Executing,18.0,1.0,18.0,1.0,0.0,1.0
2,Released,185.0,8.0,12.0,2.0,24.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()